## Data Preparation

This notebooks joins data, aligns time granularity and does some initial feature engineering for clustering.

In [89]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [90]:
cwd = os.getcwd()
DATA_DIR = os.path.join(cwd, '..', 'data')
INT_DATA_DIR = os.path.join(cwd, '..', 'data', 'interim')
RAW_DATA_DIR = os.path.join(cwd, '..', 'data', 'raw')

In [91]:
def load_data():
    """Load all required CSV files"""
    consumptions = pd.read_csv(os.path.join(RAW_DATA_DIR,'customers_consumptions.csv'))
    metadata = pd.read_csv(os.path.join(RAW_DATA_DIR,'customers_metadata.csv'))
    weather = pd.read_csv(os.path.join(RAW_DATA_DIR,'weather_data.csv'))
    prices = pd.read_csv(os.path.join(RAW_DATA_DIR,'price_data.csv'))
    
    # Convert timestamp columns to datetime
    consumptions['validfrom'] = pd.to_datetime(consumptions['validfrom'])
    weather['validfrom'] = pd.to_datetime(weather['validfrom'])
    prices['validfrom'] = pd.to_datetime(prices['validfrom'])
    
    # sort
    consumptions = consumptions.sort_values(by=["meteringpoint_id", "validfrom"])
    weather = weather.sort_values(by=["weatherstation_id", "validfrom"])
    
    return consumptions, metadata, weather, prices

In [92]:
def resample_and_ffill_prices(df, timestamp_col, freq='15T', value_cols=None):
    """
    Resample a DataFrame to a specified frequency and forward fill values.
    """
    df = df.copy()
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])
    if value_cols is None:
        value_cols = [col for col in df.columns if col != timestamp_col]
    df = df.set_index(timestamp_col)
    df_resampled = df[value_cols].resample(freq).ffill()
    df_resampled = df_resampled.reset_index()
    
    return df_resampled

In [93]:
def fill_weather_gaps(weather_df, freq='15T'):
    """
    Fill gaps in weather data by resampling and forward filling.
    """
    df = weather_df.copy()
    
    # Ensure timestamp is datetime
    df['validfrom'] = pd.to_datetime(df['validfrom'])
    value_cols = ['air_temp', 'ghi', 'cloud_opacity', 'precipitable_water']
    
    filled_dfs = []
    for station_id in df['weatherstation_id'].unique():
        station_data = df[df['weatherstation_id'] == station_id].copy()
        station_data = station_data.set_index('validfrom')
        filled_data = station_data[value_cols].resample(freq).ffill()
        filled_data = filled_data.reset_index()
        filled_data['weatherstation_id'] = station_id
        
        filled_dfs.append(filled_data)
    
    result = pd.concat(filled_dfs, ignore_index=True)
    result = result.sort_values(['weatherstation_id', 'validfrom'])
    
    return result

In [94]:
def normalise(df, group_col='meteringpoint_id'):
    """
    Normalize data using z-score within each group.
    """
    normalized = df.copy()
    
    # Columns to normalize
    cols_to_normalize = ['quantity', 'air_temp', 'ghi', 'cloud_opacity', 
                        'precipitable_water', 'price']
    
    # Normalize each feature within each group
    for col in cols_to_normalize:
        normalized[f'{col}_norm'] = (df.groupby(group_col)[col]
                                   .transform(lambda x: (x - x.mean()) / x.std()))
    
    return normalized

In [95]:
consumptions_df, metadata_df, weather_df, prices_df = load_data()

In [96]:
weather_df = fill_weather_gaps(weather_df, freq='15T')

In [97]:
prices_df = resample_and_ffill_prices(
    df=prices_df,
    timestamp_col='validfrom',
    freq='15T',
    value_cols=['price']
)

In [98]:
merged_df = (consumptions_df
 .merge(metadata_df, on=["meteringpoint_id"], how='left')
 .merge(weather_df, on=["weatherstation_id", "validfrom"], how='left')
 .merge(prices_df, on=["validfrom"], how="left")
 .sort_values(by=["meteringpoint_id", "validfrom"])
 .drop_duplicates())

In [99]:
normal_df = normalise(merged_df)

## Saving data

In [100]:
merged_df.to_csv(os.path.join(INT_DATA_DIR, 'merged.csv'), index=False)

In [101]:
normal_df.to_csv(os.path.join(INT_DATA_DIR, 'norm_merged.csv'), index=False)